In [1]:
"""I used the documentation in addition to the following video for help on this webscraping/API process:
    https://www.youtube.com/watch?v=yexxcrPC7U8 """

import requests
import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import praw
import json
import time
import datetime
import re
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from IPython.core.display import HTML

driver = webdriver.PhantomJS(executable_path='/Users/thomas/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs')
driver.set_window_size(300, 200) 

%matplotlib inline

## The below section is for reddit scraping

In [2]:
def get_replies(comment, reply_list):
    """ recursive function that goes through the reply tree of a comment """
    reply_list.append({"id": comment.id, "comment": comment.body})
    if len(comment._replies) > 0:
        for r in comment._replies:
            get_replies(r, reply_list)
    return reply_list

In [66]:
subreddit_dictionary={}
title_list=[]

###
Number_of_Posts=300
Number_of_Comments=12



subreddit_list=["technology","politics","news","worldnews","inthenews","Futurology","StockMarket"]

for i in range(5,len(subreddit_list)):
    count=0
    current_time=time.time()
    print "Starting Collection from r/"+subreddit_list[i]
    subreddit_posts={}
    json_name='/Users/thomas/GA-DSI/projects/projects-capstone/Work/r_'\
                        +subreddit_list[i]+'.json'
    info_for_request='get top '+str(Number_of_Posts)+' posts w/ top '+str(Number_of_Comments)+\
                        ' comments and children comments from r/'+subreddit_list[i]+\
                        "for Capstone project at General Assembly's DSI program by User: dingdong1111"
    r = praw.Reddit(user_agent=info_for_request)
    r.login('dingdong1111', '01001759',disable_warning=True)
    posts = r.get_subreddit(subreddit_list[i]).get_top_from_year(limit=Number_of_Posts)
    for post in posts:
        count+=1
        post_comments=[]
        
        post_id=post.id
        post_title=post.title
        post_score=post.score
        post_url=post.url
        post_date=datetime.datetime.fromtimestamp(post.created)
#         post_author=post.author
        
        submission = r.get_submission(submission_id=post_id)
        submission.replace_more_comments(limit=Number_of_Comments, threshold=0)
        for comment in submission.comments:
            """ loops through all comments to submission """
            x = []  # array that holds the comment and all replies
            # get all replies and append it to our array for writing to file
            post_comments.append(get_replies(comment, x))
            
        if count%20==0:
            print "  Finished",count,"Out of",Number_of_Posts
            print "    Time Elapsed:",(time.time()-current_time)/60.,"minutes"
        subreddit_posts[post_id]=[post_title,str(post_date),post_score,post_url,post_comments]
    with open(json_name, 'w') as fp:
        json.dump(subreddit_posts, fp)
    
    print "Time Spent:",(time.time()-current_time)/60.,"minutes"
    print "Finished Collection from r/"+subreddit_list[i]
    print
    

Starting Collection from r/Futurology
  Finished 20 Out of 300
    Time Elapsed: 8.71059374809 minutes
  Finished 40 Out of 300
    Time Elapsed: 17.3826767166 minutes
  Finished 60 Out of 300
    Time Elapsed: 26.0483352661 minutes
  Finished 80 Out of 300
    Time Elapsed: 34.7198677659 minutes
  Finished 100 Out of 300
    Time Elapsed: 43.4012860497 minutes
  Finished 120 Out of 300
    Time Elapsed: 52.1055244684 minutes
  Finished 140 Out of 300
    Time Elapsed: 60.7756651998 minutes
  Finished 160 Out of 300
    Time Elapsed: 69.4521248182 minutes
  Finished 180 Out of 300
    Time Elapsed: 78.1329490145 minutes
  Finished 200 Out of 300
    Time Elapsed: 86.803851517 minutes
  Finished 220 Out of 300
    Time Elapsed: 95.5034796675 minutes
  Finished 240 Out of 300
    Time Elapsed: 104.171271618 minutes
  Finished 260 Out of 300
    Time Elapsed: 112.703770681 minutes
  Finished 280 Out of 300
    Time Elapsed: 120.543548282 minutes
  Finished 300 Out of 300
    Time Elapsed:

In [ ]:
"""THIS IS THE CODE FROM THE AFOREMENTIONED VIDEO (SEE IMPORT CELL)"""


import praw

#key in the submission ID of the Reddit thread you want to scrape
sub_id = "xxxxxx"

def get_replies(comment, reply_list):
    """ recursive function that goes through the reply tree of a comment """
    reply_list.append({"id": comment.id, "comment": comment.body})
    if len(comment._replies) > 0:
        for r in comment._replies:
            get_replies(r, reply_list)
    return reply_list


#state your purpose for conducting the comment scrape
r = praw.Reddit('get comments from xxxxxxxx for xxxxxxx by User: xxxxxxxxx')
submission = r.get_submission(submission_id=sub_id)
""" this is a costly operation due to reddit's limits but is necessary
    to get all comments and prevent errors in script """
submission.replace_more_comments(limit=None, threshold=0)

# final array for writing to file
final = []

for comment in submission.comments:
    """ loops through all comments to submission """
    x = []  # array that holds the comment and all replies
    # get all replies and append it to our array for writing to file
    final.append(get_replies(comment, x))

with open('comments.csv', 'a') as outf:
    """ open file and write to it """
    # 2 dimension array needs two for loops
    for ele in final:
        for ele2 in ele:
            # write to file in the proper comma delimited format, one per line
            outf.write(u"{0},{1}\n".format(ele2['id'], ele2['comment']).encode('utf-8'))

## The below section is for federalreserve.gov scraping

https://www.federalreserve.gov/newsevents/press/all/2016all.htm

In [221]:
fed_dict={}

sites=["https://www.federalreserve.gov/newsevents/press/all/2016all.htm",\
       "https://www.federalreserve.gov/newsevents/press/all/2015all.htm"]
r=[]
for site in sites:
    r.append(requests.get(site))
for i in range(len(r)):
    soup=BeautifulSoup.BeautifulSoup(r[i].text)
    urls=['https://www.federalreserve.gov'+str(x.find('a')['href']) for x in soup.findAll("div",{'class':'indent'})]
    titles=[x.find('a').text for x in soup.findAll("div",{'class':'indent'})]
    dates=[str('https://www.federalreserve.gov'+str(x.find('a')['href']))[-13:-5] \
                     for x in soup.findAll("div",{'class':'indent'})]
    documents=[]
    for url in urls:
        r2=requests.get(url)
        soup=BeautifulSoup.BeautifulSoup(r2.text)
        documents.append([x.text for x in soup.findAll('p')[1:]])
    for a,b,c,d in zip(dates,titles,urls,documents):
        fed_dict[a]=[b,c,d]
with open("/Users/thomas/GA-DSI/projects/projects-capstone/Work/fed_info.json", 'w') as fp:
    json.dump(fed_dict, fp)

## The following is scraping code for S&P companies off of nasdaq.com

(https://github.com/datasets/s-and-p-500-companies/blob/master/data/constituents-financials.csv)

(http://www.nasdaq.com/symbol/mmm/historical)

(https://www.youtube.com/watch?v=Q6tLD7R-nKc)

In [114]:
site="https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents-financials.csv"
save_file='/Users/thomas/GA-DSI/projects/projects-capstone/Work/s&p_stock_info.csv'
pd.read_csv(site).to_csv(save_file)
tickers=pd.read_csv(save_file)['Symbol']
tickers=[x for x in tickers]

In [178]:
#Format is (DATE,OPEN,HIGH,LOW,CLOSE/LAST,VOLUME)
select_id='ddlTimeFrame'
select_value="1 Year"
stock_dict={}
timing_counter=1
curr_time=time.time()
for ticker in tickers:
    driver = webdriver.PhantomJS(executable_path='/Users/thomas/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs')
    driver.set_window_size(300, 200) 
    url='http://www.nasdaq.com/symbol/'+ticker.lower()+'/historical'
    driver.get(url)
    time.sleep(1)
    timedropdown=WebDriverWait(driver,5).until(lambda driver: driver.find_element_by_id(select_id))
    Select(timedropdown).select_by_visible_text(select_value)
    time.sleep(1)
    soup=BeautifulSoup.BeautifulSoup(driver.page_source)
    temp=[x.text for x in soup.findAll('td')][15:-2]
    temp_l=[]
    for i in range(int(len(temp)/6.)):
        current=i*6
        temp_l.append([temp[current],temp[current+1],temp[current+2],temp[current+3],\
                       temp[current+4],temp[current+5]])
    stock_dict[ticker]=temp_l
    time.sleep(1)
    if timing_counter%25!=0:
        print "Have completed %s of 500 stocks"%timing_counter
    if timing_counter%25==0:
        print "Have completed %s of 500 stocks"%timing_counter
        print "  Elapsed Time: %s minutes"%((time.time()-curr_time)/60.)
    timing_counter+=1
    driver.close()
    
with open("/Users/thomas/GA-DSI/projects/projects-capstone/Work/s&p_stock_prices.json", 'w') as fp:
    json.dump(stock_dict, fp)

Have completed 1 of 500 stocks
Have completed 2 of 500 stocks
Have completed 3 of 500 stocks
Have completed 4 of 500 stocks
Have completed 5 of 500 stocks
Have completed 6 of 500 stocks
Have completed 7 of 500 stocks
Have completed 8 of 500 stocks
Have completed 9 of 500 stocks
Have completed 10 of 500 stocks
Have completed 11 of 500 stocks
Have completed 12 of 500 stocks
Have completed 13 of 500 stocks
Have completed 14 of 500 stocks
Have completed 15 of 500 stocks
Have completed 16 of 500 stocks
Have completed 17 of 500 stocks
Have completed 18 of 500 stocks
Have completed 19 of 500 stocks
Have completed 20 of 500 stocks
Have completed 21 of 500 stocks
Have completed 22 of 500 stocks
Have completed 23 of 500 stocks
Have completed 24 of 500 stocks
Have completed 25 of 500 stocks
  Elapsed Time: 88.0071369171 minutes
Have completed 26 of 500 stocks
Have completed 27 of 500 stocks
Have completed 28 of 500 stocks
Have completed 29 of 500 stocks
Have completed 30 of 500 stocks
Have comple

In [180]:
r=requests.get('http://www.nasdaq.com/symbol/ford/financials?query=income-statement')